In [19]:
# This file aims at converting units that are not grammes in grammes
# we consider here that : 1 c.à.c = 5 g and 1 c.à.s = 15 grammes

In [1]:
!pip install lxml 

In [2]:
from scrapping import *
import re


In [61]:
corres = open("Poids.txt", "r")
poids=corres.readlines()
dicopoids={}
for ligne in poids:
    dicopoids[str(''.join(z for z in ligne if not z.isdigit()).replace(" \n", "").lower())]=int("".join(re.findall('\d',str(ligne))))
     

In [62]:
dicopoids

{'ail tête': 80,
 'ail gousse': 7,
 'artichaut camus': 350,
 'artichaut castel': 500,
 'artichaut poivrade': 60,
 'artichaut violet': 150,
 'asperges': 50,
 'aubergine': 400,
 'carotte': 125,
 'céleri-rave': 1000,
 'chou-fleur': 1500,
 'chou vert': 2000,
 'concombre': 200,
 'courgette': 200,
 'cresson': 200,
 'échalote': 40,
 'laitue ,\n': 250,
 'navet long': 300,
 'navet rond': 200,
 'oignon': 100,
 'poireau': 150,
 'poivron': 300,
 'pomme de terre': 120,
 'tomate': 125,
 'abricot': 55,
 'ananas': 2000,
 'banane': 150,
 'citron': 100,
 'clémentine': 70,
 'datte': 35,
 'figue noire': 50,
 'melon': 650,
 'orange': 200,
 'pamplemousse': 400,
 'pêche': 150,
 'poire': 120,
 'pomme': 175,
 'olive': 4,
 'câpre .': 5}

In [54]:
url_test = "https://www.marmiton.org/recettes/recette_moelleux-au-chocolat-sans-beurre-sans-sucre_14748.aspx"

test=find_all_recipes("Steak Tartare")



100%|██████████| 15/15 [00:03<00:00,  4.09it/s]


In [92]:
test.items()

dict_items([('Steak tartare', {'moutarde de Dijon': '1c.à.c', 'oignon': '1c.à.s', 'câpres': '1c.à.s', 'sauce worcestershire': '1c.à.c', 'ketchup': '1c.à.s', 'tabasco': '', 'poivre': '', 'sel': '', "huile d'olive": '2c.à.s', 'persil': '1c.à.c', 'boeuf': '250g', "jaune d'oeuf": '1'}), ('Steak tartare arrangé', {'huile': '450g', 'moutarde': '', 'ketchup': '', 'tabasco': '', 'poivre': '', 'sel': '', 'câpres': '', 'persil': '', 'boeuf haché': '', 'oeuf': '1', 'échalote': '1', 'cornichons': ''}), ('Steak tartare végétarien', {'carotte': '500g', 'câpres': '2c.à.s', 'poivron rouge': '1', 'oignon': '1', 'persil': '1c.à.s', 'moutarde': '2c.à.s', 'tabasco': '1trait', 'poivre': '', 'sel': '', 'mayonnaise': '3c.à.s', 'sauce anglaise': '1c.à.s'}), ('Sauce légère et relevée pour steak tartare', {'moutarde de Dijon': '1c.à.s', 'fromage blanc': '6cuillères', 'poivre': '', 'sel': '', 'câpres': '1c.à.c', 'ciboulette': '', 'échalote': '1', 'mayonnaise': ''}), ("Steak tartare à l'italienne : tomate séchée 

In [41]:
"échalote"+"s"

'échalotes'

In [63]:
def convert(dico):
    for x,y in dico.items():
        for k,v in y.items():
            
            if v != '':
                n=int(("".join(re.findall('\d',str(v)))))
                
                l=str(''.join(z for z in v if not z.isdigit()))
                
                if l=='c.à.s rase':
                    y.update({k:str(15*n)+"g"})
                elif k=="jaune d'oeuf":
                    y.update({k:str(20*n)+"g"})
                elif k=="oeuf":
                    y.update({k:str(55*n)+"g"})
                elif l=='c.à.c':
                    y.update({k:str(5*n)+"g"})
                elif l=='c.à.s':
                    y.update({k:str(15*n)+"g"})
                elif l=='cuillères':
                    y.update({k:str(15*n)+"g"})
                elif v=="" and k=="sel" or k=="poivre":
                    y.update({k:str(0.5)+"g"})
                for a,b in dicopoids.items():
                    c=a+'s' #sert à prendre en compte un potentiel pluriel
                    if a in k:
                        y.update({k:str(b*n)+"g"})
            
                
            
            

In [64]:
convert(test)
test

{'Steak tartare': {'moutarde de Dijon': '5g',
  'oignon': '100000000g',
  'câpres': '15g',
  'sauce worcestershire': '5g',
  'ketchup': '15g',
  'tabasco': '',
  'poivre': '',
  'sel': '',
  "huile d'olive": '120g',
  'persil': '5g',
  'boeuf': '250g',
  "jaune d'oeuf": '160000g'},
 'Steak tartare arrangé': {'huile': '450g',
  'moutarde': '',
  'ketchup': '',
  'tabasco': '',
  'poivre': '',
  'sel': '',
  'câpres': '',
  'persil': '',
  'boeuf haché': '',
  'oeuf': '9150625g',
  'échalote': '2560000g',
  'cornichons': ''},
 'Steak tartare végétarien': {'carotte': '122070312500g',
  'câpres': '30g',
  'poivron rouge': '8100000000g',
  'oignon': '100000000g',
  'persil': '15g',
  'moutarde': '30g',
  'tabasco': '1trait',
  'poivre': '',
  'sel': '',
  'mayonnaise': '45g',
  'sauce anglaise': '15g'},
 'Sauce légère et relevée pour steak tartare': {'moutarde de Dijon': '15g',
  'fromage blanc': '90g',
  'poivre': '',
  'sel': '',
  'câpres': '5g',
  'ciboulette': '',
  'échalote': '256000

In [33]:
for x in test:
        for k,v in test.items():
            for x,y in v.items():
                print(y)
           
            

5g
15g
15g
5g
15g



30g
5g
250g
400g
450g








3025g
1

500g
30g
1
1
15g
30g
1trait


45g
15g
15g
90g


5g

1

250g
30g
15g
15g
20
7
4
50g



400g
300g
5g



2
2
150g
5g
5g
1⁄4
1petite gousse

1⁄4
3025g
1

20g
20g


4
4
16
4
300g
30g
30g
0.5g


1
15g
300g

1
15g
6



200g



600g
50g
15g
2
2
2
4
30g
4
4

15g
105g


1pavé
1
1
150g
1⁄2
2
2
2tranches
2feuilles
5g
5g
1
5g
2
3025g
2
1bouquet
45g



4
1bouquet
5g
5g
15g
15g
5g
15g



30g
5g
250g
400g
450g








3025g
1

500g
30g
1
1
15g
30g
1trait


45g
15g
15g
90g


5g

1

250g
30g
15g
15g
20
7
4
50g



400g
300g
5g



2
2
150g
5g
5g
1⁄4
1petite gousse

1⁄4
3025g
1

20g
20g


4
4
16
4
300g
30g
30g
0.5g


1
15g
300g

1
15g
6



200g



600g
50g
15g
2
2
2
4
30g
4
4

15g
105g


1pavé
1
1
150g
1⁄2
2
2
2tranches
2feuilles
5g
5g
1
5g
2
3025g
2
1bouquet
45g



4
1bouquet
5g
5g
15g
15g
5g
15g



30g
5g
250g
400g
450g








3025g
1

500g
30g
1
1
15g
30g
1trait


45g
15g
15g
90g


5g

1

250g
30g
15g
15g
20
7
4
50g



400g
300g
5g



2
2
150g

In [115]:
convert(test)

ValueError: too many values to unpack (expected 2)

In [103]:
test="erf456 efef56"
int(''.join(re.findall('\d',test)))



45656